In [1]:
import json
def get_params(filePath):
    with open(filePath, 'r') as f:
        params = json.load(f)
    f.close()
    return params

In [6]:
train_params = get_params("./params/train_params.json")
leak_gan_params = get_params("./params/leak_gan_params.json")
target_params = get_params("./params/target_params.json")
dis_data_params = get_params("./params/dis_data_params.json")
real_data_params = get_params("./params/real_data_params.json")
print( "train_params:" ,train_params,"\n\n",
        "leak_gan_params:" ,leak_gan_params,"\n\n",
        "target_params:" ,target_params,"\n\n",
        "dis_data_params:" ,dis_data_params,"\n\n",
        "real_data_params:" ,real_data_params
)

train_params: {'lr_dict': {'worker': 0.0015, 'manager': 0.0015, 'discriminator': 5e-05}, 'decay_step_size': 200, 'decay_rate': 0.99, 'total_epoch': 800, 'generated_num': 156, 'save_num': 10, 'replace_num': 5, 'pre_dis_epoch_num': 50, 'pre_gen_epoch_num': 80, 'pos_filepath': './data/train_corpus.npy', 'neg_filepath': './data/gen_data.npy', 'eval_filepath': './data/eval_data.npy', 'model_path': './ckpts/', 'seed': 233, 'checkpoint_path': None} 

 leak_gan_params: {'discriminator_params': {'seq_len': 20, 'num_classes': 2, 'vocab_size': 5258, 'dis_emb_dim': 64, 'filter_sizes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20], 'num_filters': [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160], 'start_token': 0, 'goal_out_size': None, 'step_size': 5, 'dropout_prob': 0.8, 'l2_reg_lambda': 0.2}, 'generator_params': {'manager_params': {'batch_size': 64, 'hidden_dim': 32, 'goal_out_size': None}, 'worker_params': {'batch_size': 64, 'vocab_size': 5258, 'embed_dim': 32, 'hidden_dim': 32, 'goal_out_

In [2]:
train_params = get_params("./params/train_params.json")
leak_gan_params = get_params("./params/leak_gan_params.json")
target_params = get_params("./params/target_params.json")
dis_data_params = get_params("./params/dis_data_params.json")
real_data_params = get_params("./params/real_data_params.json")
print( "train_params:" ,train_params,"\n\n",
        "leak_gan_params:" ,leak_gan_params,"\n\n",
        "target_params:" ,target_params,"\n\n",
        "dis_data_params:" ,dis_data_params,"\n\n",
        "real_data_params:" ,real_data_params
)

train_params: {'lr_dict': {'worker': 0.0015, 'manager': 0.0015, 'discriminator': 5e-05}, 'decay_step_size': 200, 'decay_rate': 0.99, 'total_epoch': 800, 'generated_num': 156, 'save_num': 10, 'replace_num': 5, 'pre_dis_epoch_num': 50, 'pre_gen_epoch_num': 80, 'pos_filepath': './data/train_corpus.npy', 'neg_filepath': './data/gen_data.npy', 'eval_filepath': './data/eval_data.npy', 'model_path': './ckpts/', 'seed': 233, 'checkpoint_path': None} 

 leak_gan_params: {'discriminator_params': {'seq_len': 20, 'num_classes': 2, 'vocab_size': 5258, 'dis_emb_dim': 64, 'filter_sizes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20], 'num_filters': [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160], 'start_token': 0, 'goal_out_size': None, 'step_size': 5, 'dropout_prob': 0.8, 'l2_reg_lambda': 0.2}, 'generator_params': {'manager_params': {'batch_size': 64, 'hidden_dim': 32, 'goal_out_size': None}, 'worker_params': {'batch_size': 64, 'vocab_size': 5258, 'embed_dim': 32, 'hidden_dim': 32, 'goal_out_

In [6]:
negative_filepath = './data/gen_corpus.npy'
import numpy as np
gen_corpus = np.load(negative_filepath)

In [7]:
gen_corpus.size

199680

In [8]:
gen_corpus[1]

array([ 319, 2341,  902,  986,  197, 5170, 2609, 1148, 3956, 4507, 1842,
       4939, 1574, 2646, 4273, 4778, 3695, 2925, 2212, 4570], dtype=int64)

In [9]:
gen_corpus

array([[2036, 4029, 1620, ...,  917,  268,  260],
       [ 319, 2341,  902, ..., 2925, 2212, 4570],
       [4904,  189,  688, ...,  671,  416,   13],
       ...,
       [ 717, 3149, 4776, ..., 1957, 4663, 1243],
       [5117, 1407, 2671, ..., 2175, 3971, 2533],
       [1477, 1585,  942, ..., 2756, 3729,  411]], dtype=int64)

In [10]:
len(gen_corpus)

9984

In [15]:
import torch.nn as nn

seq_len= 20
num_classes= 2 
vocab_size= 5258 
dis_emb_dim= 64 
filter_sizes= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]
num_filters= [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160]
start_token= 0
goal_out_size= sum(num_filters)
step_size= 5
dropout_prob= 0.8
l2_reg_lambda= 0.2

class Discriminator(nn.Module):
    def __init__(self, seq_len, num_classes, vocab_size, dis_emb_dim, 
                    filter_sizes, num_filters, start_token, goal_out_size, step_size, dropout_prob, l2_reg_lambda):
      
        super(Discriminator, self).__init__()
        self.seq_len = seq_len
        self.num_classes = num_classes
        self.vocab_size = vocab_size
        self.dis_emb_dim = dis_emb_dim
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters
        self.start_token = start_token
        self.goal_out_size = goal_out_size
        self.step_size = step_size
        self.dropout_prob = dropout_prob
        self.l2_reg_lambda = l2_reg_lambda
        self.num_filters_total = sum(self.num_filters)
        
        #Building up layers
        self.emb = nn.Embedding(self.vocab_size + 1, self.dis_emb_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_f, (f_size, self.dis_emb_dim)) for f_size, num_f in zip(self.filter_sizes, self.num_filters)
        ])
        self.highway = nn.Linear(self.num_filters_total, self.num_filters_total)
        #in_features = out_features = sum of num_festures
        self.dropout = nn.Dropout(p = self.dropout_prob)
        #Randomly zeroes some of the elements of the input tensor with probability p using Bernouli distribution
        #Each channel will be zeroed independently onn every forward call
        self.fc = nn.Linear(self.num_filters_total, self.num_classes)
        
    def forward(self, x):
        """
        Argument:
            x: shape(batch_size * self.seq_len)
               type(Variable containing torch.LongTensor)
        Return:
            pred: shape(batch_size * 2)
                  For each sequence in the mini batch, output the probability
                  of it belonging to positive sample and negative sample.
            feature: shape(batch_size * self.num_filters_total)
                     Corresponding to f_t in original paper
            score: shape(batch_size, self.num_classes)
              
        """
        #1. Embedding Layer
        #2. Convolution + maxpool layer for each filter size
        #3. Combine all the pooled features into a prediction
        #4. Add highway
        #5. Add dropout. This is when feature should be extracted
        #6. Final unnormalized scores and predictions
        emb = self.emb(x).unsqueeze(1)
        convs = [F.relu(conv(emb)).squeeze(3) for conv in self.convs] # [batch_size * num_filter * seq_len]
        pooled_out = [F.max_pool1d(conv, conv.size(2)).squeeze(2) for conv in convs] # [batch_size * num_filter]
        pred = torch.cat(pooled_out, 1) # batch_size * sum(num_filters)
        #print("Pred size: {}".format(pred.size()))
        highway = self.highway(pred)
        #print("highway size: {}".format(highway.size()))
        highway = torch.sigmoid(highway)* F.relu(highway) + (1.0 - torch.sigmoid(highway))*pred
        features = self.dropout(highway)
        score = self.fc(features)
        pred = F.log_softmax(score, dim=1) #batch * num_classes
        return {"pred":pred, "feature":features, "score": score}

In [17]:
D = Discriminator(seq_len, num_classes, vocab_size, dis_emb_dim, filter_sizes, num_filters, start_token, goal_out_size, step_size, dropout_prob, l2_reg_lambda)

In [18]:
D

Discriminator(
  (emb): Embedding(5259, 64)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(1, 64), stride=(1, 1))
    (1): Conv2d(1, 200, kernel_size=(2, 64), stride=(1, 1))
    (2): Conv2d(1, 200, kernel_size=(3, 64), stride=(1, 1))
    (3): Conv2d(1, 200, kernel_size=(4, 64), stride=(1, 1))
    (4): Conv2d(1, 200, kernel_size=(5, 64), stride=(1, 1))
    (5): Conv2d(1, 100, kernel_size=(6, 64), stride=(1, 1))
    (6): Conv2d(1, 100, kernel_size=(7, 64), stride=(1, 1))
    (7): Conv2d(1, 100, kernel_size=(8, 64), stride=(1, 1))
    (8): Conv2d(1, 100, kernel_size=(9, 64), stride=(1, 1))
    (9): Conv2d(1, 100, kernel_size=(10, 64), stride=(1, 1))
    (10): Conv2d(1, 160, kernel_size=(15, 64), stride=(1, 1))
    (11): Conv2d(1, 160, kernel_size=(20, 64), stride=(1, 1))
  )
  (highway): Linear(in_features=1720, out_features=1720, bias=True)
  (dropout): Dropout(p=0.8, inplace=False)
  (fc): Linear(in_features=1720, out_features=2, bias=True)
)

In [12]:
import torch.nn as nn
import torch
import numpy as np
conv2d_1 = nn.Conv2d(1, 100, (1, 64))
new_array = torch.from_numpy(np.random.randint(100 , size = (20,64)))
print("new_array: {}".format(new_array.size()))
new_input = new_array.unsqueeze(0).unsqueeze(0)
print("new_input: {}".format(new_input.size()))
new_output = conv2d_1(torch.IntTensor(new_input))
print("new_output: {}".format(new_output.size()))

new_array: torch.Size([20, 64])
new_input: torch.Size([1, 1, 20, 64])


RuntimeError: Expected object of scalar type Int but got scalar type Float for argument #3 'mat1' in call to _th_addmm_

In [15]:
input_2 = torch.randn(1, 1, 20, 64)
output_1 = conv2d_1(input_2)
print("input_2: {}".format(input_2.size()))
print("output_1: {}".format(output_1.size()))

input_2: torch.Size([1, 1, 20, 64])
output_1: torch.Size([1, 100, 20, 1])


In [17]:
conv2d_2 = nn.Conv2d(1, 200, kernel_size=(2, 64), stride=(1, 1))
output_2 = conv2d_2(input_2)
print("input_2: {}".format(input_2.size()))
print("output_2: {}".format(output_2.size()))

input_2: torch.Size([1, 1, 20, 64])
output_2: torch.Size([1, 200, 19, 1])
